In [77]:
import numpy as np
from numpy.linalg import inv

In [78]:
x_observations = np.array([4000, 4260, 4550, 4860, 5110])
v_observations = np.array([280, 282, 285, 286, 290])

In [79]:
z = np.c_[x_observations, v_observations]
z

array([[4000,  280],
       [4260,  282],
       [4550,  285],
       [4860,  286],
       [5110,  290]])

In [80]:
# Initial Conditions
a = 2  # Acceleration
v = 280
t = 1  # Difference in time

# Process / Estimation Errors
error_est_x = 20
error_est_v = 5

# Observation or Measurement Errors
error_obs_x = 25  # Uncertainty in the measurement
error_obs_v = 6


In [81]:
def prediction2d(x, v, t, a):
    A = np.array([[1, t],
                  [0, 1]])
    X = np.array([[x],
                  [v]])
    B = np.array([[0.5 * t ** 2],
                  [t]])
    X_prime = A.dot(X) + B.dot(a)
    return X_prime

In [82]:
def covariance2d(sigma1, sigma2):
    cov1_2 = sigma1 * sigma2
    cov2_1 = sigma2 * sigma1
    cov_matrix = np.array([[sigma1 ** 2, cov1_2],
                           [cov2_1, sigma2 ** 2]])
    return cov_matrix

In [83]:
covariance_matrix = covariance2d(error_est_x, error_est_v)

### P : Estimation Covariance matrix
### A : State Transition Matrix

#### P with error terms that correspond to the variance of the x position and x velocity, specific to the estimates.

In [84]:
# Initial Estimation Covariance Matrix with zeroing off non-diagnols values
P=np.diag(np.diag(covariance_matrix))
P

array([[400,   0],
       [  0,  25]])

In [85]:
A = np.array([[1, t],
              [0, 1]])
A

array([[1, 1],
       [0, 1]])

### Initial State Matrix : It contains the inital value of distance and velocity

In [86]:
X = np.array([[z[0][0]],
              [v]])
X

array([[4000],
       [ 280]])

In [87]:
n = len(z[0])
n

2

In [88]:
for data in z[0:]:
    print(data)

[4000  280]
[4260  282]
[4550  285]
[4860  286]
[5110  290]


In [89]:
X[0][0], X[1][0]

(4000, 280)

In [93]:
H = np.identity(n)
H

array([[1., 0.],
       [0., 1.]])

#### K: Kalman Gain
#### P: Process Covariance Matrix
#### Y: Matrix with measurement data

In [90]:
for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    P = np.diag(np.diag(A.dot(P).dot(A.T)))
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))
#     print(K)
    Y = H.dot(data).reshape(n, -1)
    X = X + K.dot(Y - H.dot(X))
    print(X)
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

[[4267.89711191]
 [ 283.89530686]]
[[4552.90991661]
 [ 285.69837999]]
[[4856.53383383]
 [ 285.16812012]]
[[5115.48736328]
 [ 291.31696719]]


In [92]:

for data in z[1:]:
    X = prediction2d(X[0][0], X[1][0], t, a)
    # To simplify the problem, set off-diagonal terms to 0.
    P = np.diag(np.diag(A.dot(P).dot(A.T)))

    # Calculating the Kalman Gain
    H = np.identity(n)
    R = covariance2d(error_obs_x, error_obs_v)
    S = H.dot(P).dot(H.T) + R
    K = P.dot(H).dot(inv(S))

    # Reshape the new data into the measurement space.
    Y = H.dot(data).reshape(n, -1)

    # Update the State Matrix
    # Combination of the predicted state, measured values, covariance matrix and Kalman Gain
    X = X + K.dot(Y - H.dot(X))

    # Update Process Covariance Matrix
    P = (np.identity(len(K)) - K.dot(H)).dot(P)

print("Kalman Filter State Matrix:\n", X)
 

('Kalman Filter State Matrix:\n', array([[5701.95695768],
       [ 432.06966984]]))
